In [196]:
%serialconnect to --port="COM5" --baud=115200

Connecting to --port=COM5 --baud=115200 
Traceback (most recent call last):
  File "main.py", line 29, in <module>
  File "main.py", line 29, in <dictcomp>
KeyboardInterrupt: 
MicroPython 94a9b50-dirty on 2022-03-20; ESP32 module with ESP32
Type "help()" for more information.
>>> 
>>> 
raw REPL; CTRL-B to exit
>Ready.


In [150]:
%sendtofile lib/runner.py --source micropython/lib/runner.py

Sent 248 lines (8271 bytes) to lib/runner.py.


In [197]:
print(gc.mem_free())
gc.collect()

84640


In [205]:
print('before import:', gc.mem_free())

from lib.runner import Runner
from lib.utils import connect_wifi
from lib.requests import MicroWebCli as requests

print('after import:', gc.mem_free())

ssid = 'TP-Link_AP_4C04'
password = '63525465'
connect_wifi(ssid, password)

print('after wifi:', gc.mem_free())

Nc = 1
Ns = 50

# stim_freqs = [8, 10, 12]
stim_freqs = [8, 10, 12]
# Here, we select the algorithm. Can be one of ['MsetCCA', 'GCCA', 'CCA']
decoding_algo = 'MsetCCA'
# decoding_algo = 'GCCA'

print('before setup:', gc.mem_free())

runner = Runner(decoding_algo, buffer_size=Ns, stimulus_freqs=stim_freqs) # initialise a base runner
runner.setup() # setup peripherals and memory buffers

print('after import:', gc.mem_free())

gc.collect()

calibration_data = {}

print('before second import:', gc.mem_free())

import ujson as json
from ulab import numpy as np

print('after second import:', gc.mem_free())

number = 1

data = requests.GETRequest("http://192.168.0.37:5001/calib_{n}_7".format(n=number))
calibration_data[8] = np.array(json.loads(data)['7'])

gc.collect()

data = requests.GETRequest("http://192.168.0.37:5001/calib_{n}_10".format(n=number))
calibration_data[10] = np.array(json.loads(data)['10'])

gc.collect()

data = requests.GETRequest("http://192.168.0.37:5001/calib_{n}_12".format(n=number))
calibration_data[12] = np.array(json.loads(data)['12'])

print('before delete import:', gc.mem_free())

del json
del np
gc.collect()

print('before calibration:', gc.mem_free())

runner.calibrate(calibration_data)

print('after calibration:', gc.mem_free())

# calibration_data = #import json saved on ROM
# runner.calibrate(calibration_data)
gc.collect()

before import: 50352
after import: 50304
network config: ('192.168.0.88', '255.255.255.0', '192.168.0.1', '194.168.4.100')
after wifi: 50144
before setup: 50112
ADC initialised
SPI initialised
DigiPot set to 255 = gain of 17.57861814046062
after import: 54736
before second import: 54848
after second import: 54832
before delete import: 36352
before calibration: 54864
after calibration: 13392


In [206]:
from ulab import numpy as np
import ujson as json

#8hz
count8 = 0
for count in range(26):
    test = requests.GETRequest("http://192.168.0.37:5001/test_7_{n}".format(n=number),{'count':count})
    test = np.array(json.loads(test)['test'])
    result = runner.decoder.classify(test)
    if max(result, key=result.get) == 8:
        count8+=1

gc.collect()
count10 = 0
for count in range(26):
    test = requests.GETRequest("http://192.168.0.37:5001/test_10_{n}".format(n=number),{'count':count})
    test = np.array(json.loads(test)['test'])
    result = runner.decoder.classify(test)
    if max(result, key=result.get) == 10:
        count10+=1
        
gc.collect()
count12 = 0
for count in range(26):
    test = requests.GETRequest("http://192.168.0.37:5001/test_12_{n}".format(n=number),{'count':count})
    test = np.array(json.loads(test)['test'])
    result = runner.decoder.classify(test)
    if max(result, key=result.get) == 12:
        count12+=1
        
gc.collect()
print(count8/26,count10/26,count12/26)
# print(count8/26,count12/26)

....0.6153846153846154 0.6153846153846154 0.7307692307692307
